load all the testSet preprocessed in create_data

for one data

- data[0] is the identity in BabelNet
- data[1] is list of definitions, standing for different senses
- data[2] is list of tree sequences, each nodes is a sememe
- data[3] is list of matrix of distance of different nodes, using for `TaSTG model`
- data[4] is list of depth of nodes, using for `Tree Attn`


In [16]:

import torch
from torchvision import transforms
from imp import reload

from create_data import sememeDataset
import tree2seq


testSet = torch.load("./data/testSet.pkl")

print(testSet[4122])


['bn:00107165a', ['Connected with or belonging to or used in a navy', 'Of or relating to a navy.'], [['start', 'part|部件', 'military|军', 'end', 'army|军队', 'end', 'fight|争斗', '~', 'end', 'waters|水域', 'end', 'end', 'end']], [tensor([[[0, 2, 4, 3, 4, 3, 4, 6, 5, 6, 5, 3],
         [2, 0, 2, 1, 2, 1, 2, 4, 3, 4, 3, 1],
         [4, 2, 0, 1, 4, 3, 4, 6, 5, 6, 5, 3],
         [3, 1, 1, 0, 3, 2, 3, 5, 4, 5, 4, 2],
         [4, 2, 4, 3, 0, 1, 4, 6, 5, 6, 5, 3],
         [3, 1, 3, 2, 1, 0, 3, 5, 4, 5, 4, 2],
         [4, 2, 4, 3, 4, 3, 0, 2, 1, 2, 1, 1],
         [6, 4, 6, 5, 6, 5, 2, 0, 1, 4, 3, 3],
         [5, 3, 5, 4, 5, 4, 1, 1, 0, 3, 2, 2],
         [6, 4, 6, 5, 6, 5, 2, 4, 3, 0, 1, 3],
         [5, 3, 5, 4, 5, 4, 1, 3, 2, 1, 0, 2],
         [3, 1, 3, 2, 3, 2, 1, 3, 2, 3, 2, 0]]])], [tensor([[0, 2, 4, 3, 4, 3, 4, 6, 5, 6, 5, 3]])]]


now we can try to look at one sememe tree. like this: 

In [17]:
reload(tree2seq)

treeSeq = testSet[4122][2][0]
print(f"tree sequence: {treeSeq}")

tree = tree2seq.seq2tree(treeSeq)
print(tree)

tree sequence: ['start', 'part|部件', 'military|军', 'end', 'army|军队', 'end', 'fight|争斗', '~', 'end', 'waters|水域', 'end', 'end', 'end']
start
└── part|部件
    ├── army|军队
    ├── fight|争斗
    │   ├── waters|水域
    │   └── ~
    └── military|军



then, we can try to compute the `strict,edge,vertex f1` for the two example
for the exact same tree:

In [18]:
treeSeqOther = ['start', 'part|部件', 'military|军', 'end', 'army|军队', 'end', 'fight|争斗', '~', 'end', 'waters|水域', 'end', 'end', 'end']

score = tree2seq.computeStrict(treeSeq, treeSeqOther, endtoken="end")
print(f"strict: {score[0][0]}, edge: {score[0][1]}, vertex: {score[0][2]}")

strict: 1.0, edge: 1.0, vertex: 1.0


but if we change only the first meaningful node
- strict F1 go to 0, because it doesn't consider the subtree if the root is miss
- edge F1 consider whether the branch is correct(`father=father && child=child`)
- vertex F1 is high, because these two trees have most vertex the same `except attack_sememe`

under the ecumenic condition: `strict <= edge <= vertex`

In [19]:
treeSeqOtherChange = ['start', 'attack_sememe', 'military|军', 'end', 'army|军队', 'end', 'fight|争斗', '~', 'end', 'waters|水域', 'end', 'end', 'end']

score = tree2seq.computeStrict(treeSeq, treeSeqOtherChange, endtoken="end")
print(f"strict: {score[0][0]}, edge: {score[0][1]}, vertex: {score[0][2]}")

strict: 0.0, edge: 0.3333333333333333, vertex: 0.8333333333333334
